In [131]:
import pandas as pd
import os

In [132]:
output_directory = "/home/nina/PycharmProjects/memilio/analysis/systems_files"

In [133]:
file_directory = "/home/nina/PycharmProjects/memilio/analysis/systems_files/Euler/"
sampling_files = [el for el in os.listdir(file_directory) if "Sampling_COV_INSIDe" in el]
sampling_curves_files = [el for el in os.listdir(file_directory) if (("INSIDe_substances_results" in el) and ("Sampling" not in el))]
sampling_curves_files

['INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW046.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW017.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_RW157.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW048.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW059.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_RW211.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_RW126.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW023.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW061.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW054.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_RW141.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW064.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW043.txt',
 'INSIDe_substances_results_2023_08_30__11_38_37_manhole_MW052.txt',
 'INSIDe_substances_results_2023_0

In [139]:
def tidy_concentration_curve(path):
    # load the complete dataset
    df = pd.read_csv(path, sep="\t")
    df.rename({"time[min]": "time"}, axis=1, inplace=True)

    sampling_location = path.split("_manhole_")[1].replace(".txt", "")

    # identify positions of new subdatasets
    subdataframes = df.loc[df["time"]=="time[min]"]

    # CSB (Chemischer Sauerstoffbedarf) concentration
    df_1 = df.iloc[:subdataframes.index[0]].copy()
    df_1["location"] = sampling_location

    # COV(COV19) concentration 
    df_2 = df.iloc[subdataframes.index[0]:subdataframes.index[1]].copy()
    df_2.columns = df_2.iloc[0]
    df_2 = df_2.iloc[1:]
    df_2.reset_index(drop=True, inplace=True)
    df_2.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_2["location"] = sampling_location

    # T(Temperature) concentration
    df_3 = df.iloc[subdataframes.index[1]:subdataframes.index[2]].copy()
    df_3.columns = df_3.iloc[0]
    df_3 = df_3.iloc[1:]
    df_3.reset_index(drop=True, inplace=True)
    df_3.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_3["location"] = sampling_location

    df_4 = df.iloc[subdataframes.index[2]:].copy()
    df_4.columns = df_4.iloc[0]
    df_4 = df_4.iloc[1:]
    df_4.reset_index(drop=True, inplace=True)
    df_4.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_4["location"] = sampling_location

    # combine the dataframes
    df_1[df_2.columns[1]] = df_2.iloc[:,1]
    df_1[df_3.columns[1]] = df_3.iloc[:,1]
    df_1[df_4.columns[1]] = df_4.iloc[:,1]
    return df_1


In [140]:
df = pd.DataFrame()
for i in range(len(sampling_curves_files)):
    path = f"{file_directory}{sampling_curves_files[i]}"
    df_sub = tidy_concentration_curve(path)
    df = pd.concat((df, df_sub))
df.reset_index(inplace=True, drop=True)

In [141]:
df

,time,CSB(Chemischer Sauerstoffbedarf) concentration [umol/l],location,AFS63(Abfiltrierbare Stoffe) concentration [umol/l],COV(COV19) concentration [umol/l],T(Temperatur) concentration [umol/l]
0,-10,0.0,MW046,0.0,0.0,0.0
1,-5,0.0,MW046,0.0,0.0,0.0
2,0,0.0,MW046,0.0,0.0,0.0
3,5,107.0,MW046,728.2,3.3,0.0
4,10,107.0,MW046,727.7,0.3,0.0
...,...,...,...,...,...,...
5449,1480,107.0,RW156,727.5,0.4,0.0
5450,1485,107.0,RW156,727.5,0.4,0.0
5451,1490,107.0,RW156,727.4,0.4,0.0
5452,1495,107.0,RW156,727.4,0.4,0.0


In [146]:
df.columns = ['time', 'CSB',
       'location', 'AFS63',
       'COV19',
       'Temperature']

"""
['time', 'CSB(Chemischer Sauerstoffbedarf) concentration [umol/l]',
       'location', 'AFS63(Abfiltrierbare Stoffe) concentration [umol/l]',
       'COV(COV19) concentration [umol/l]',
       'T(Temperatur) concentration [umol/l]']
"""

"\n['time', 'CSB(Chemischer Sauerstoffbedarf) concentration [umol/l]',\n       'location', 'AFS63(Abfiltrierbare Stoffe) concentration [umol/l]',\n       'COV(COV19) concentration [umol/l]',\n       'T(Temperatur) concentration [umol/l]']\n"

In [147]:
df["location"].unique()

array(['MW046', 'MW017', 'RW157', 'MW048', 'MW059', 'RW211', 'RW126',
       'MW023', 'MW061', 'MW054', 'RW141', 'MW064', 'MW043', 'MW052',
       'RW143', 'RW155', 'MW022', 'RW156'], dtype=object)

In [148]:
df.to_csv(output_directory+"/concentration_curves.csv", index=False)

In [149]:
path

'/home/nina/PycharmProjects/memilio/analysis/systems_files/Euler/INSIDe_substances_results_2023_08_30__11_38_37_manhole_RW156.txt'